# 0. Standard imports

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/Users/khalilouardini/Desktop/projects/scVI/scvi')

In [2]:
import ete3
import pickle
import networkx as nx
import pandas as pd
import pickle

# 1. Importing a tree

***Converting tree from networkx format to ete3***

In [15]:
tree_file = "/Users/khalilouardini/Desktop/projects/scVI/scvi/data/1000cells/med_fitness/topology20.pkl"

with open(tree_file, 'rb') as f:
    graph = pickle.load(f)

# adding branch length attribute to the leaves
for n in graph.nodes(data=True):
    if type(n[0]) == int:
        n[1]['branch_length'] = 0
    else:
        n[1]['branch_length'] = n[0].branch_length

In [16]:
def networkx_to_ete3(graph, root):
    subtrees = {node[0]: ete3.Tree(name=node[0], dist=node[1]['branch_length']) for node in graph.nodes(data=True)}
    [*map(lambda edge: subtrees[edge[0]].add_child(subtrees[edge[1]]), graph.edges())]
    tree = subtrees[root]
    return tree

In [17]:
root = [n for n, d in graph.in_degree() if d==0] [0]
tree = networkx_to_ete3(graph, root)
tree

Tree node 'Clade' (0x12470ab9)

In [18]:
len(tree.get_leaves())

1000

In [19]:
print(tree)


               /-89
              |
            /-|   /-91
           |  |  |
           |   \-|      /-104
           |     |   /-|
           |     |  |   \-106
           |      \-|
           |        |   /-109
           |         \-|
           |            \-110
           |
           |            /-52
           |         /-|
           |        |   \-61
         /-|      /-|
        |  |     |  |   /-65
        |  |     |   \-|
        |  |   /-|      \-83
        |  |  |  |
        |  |  |  |   /-22
        |  |  |  |  |
        |  |  |   \-|   /-27
        |  |  |     |  |
        |  |  |      \-|   /-32
        |  |  |        |  |
        |  |  |         \-|   /-33
        |  |  |           |  |
      /-|   \-|           |  |      /-34
     |  |     |            \-|   /-|
     |  |     |              |  |  |   /-41
     |  |     |              |  |   \-|
     |  |     |               \-|      \-42
     |  |     |                 |
     |  |     |                 |   /-47


***Visualizing on iTol***

In [20]:
#import sys
#sys.path.append('/Users/khalilouardini/Desktop/projects/scVI/scvi/notebooks/plots')

#import pandas as pd
#import plot_tree_itol, itol_manager

re-indexing + saving branch length data

In [21]:
branch_lengths = {}
node_names = []

for i, n in enumerate(tree.traverse('levelorder')):
    #n.add_features(index=i)
    n.name = i
    node_names.append(str(i))
    branch_lengths[n.name] = n.dist
    
df = pd.DataFrame(list(branch_lengths.values()),
                  index = node_names, 
                  columns =['branch_length']
                 )

df.to_csv('/Users/khalilouardini/Desktop/projects/scVI/scvi/notebooks/plots/richards-vanilla.txt',
          sep='\t'
         )

df.head(50)

,branch_length
0,0.148379
1,0.238553
2,0.058744
3,0.005382
4,0.005388
5,0.086639
6,0.002652
7,0.063141
8,0.000000
9,0.003951


Upload tree

In [ ]:
#apiKey = 'GMkmiUxXIyzqip9q2YxVyg'
#projectName = 'CasVI'
#tree_out = "Richard-Vanilla"

annotations files

In [ ]:
#files = []
#files += [itol_manager.create_annotation_file_for_itol(tree,
#                                                       df,
#          '/Users/khalilouardini/Desktop/projects/scVI/scvi/notebooks/plots/richards_vanilla.txt')]

In [ ]:
#plot_tree_itol.upload_to_itol(tree, apiKey, projectName, 
#                              files = files,
#                              outfp = tree_out,
#                              tree_name = 'richards-vanilla')

# 2. GLM simulation

In [8]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA 
from scvi.dataset.poisson_glm import Poisson_GLM
from utils.plots_util import plot_losses, plot_elbo, plot_common_ancestor, plot_one_gene

ModuleNotFoundError: No module named 'utils'

In [ ]:
import numpy as np
import pandas as pd

from scvi.dataset.anndataset import AnnDatasetFromAnnData
from scvi.models import *;
from scvi.utils.data_util import get_leaves, get_internal, get_leaves
from scvi.utils.precision_matrix import precision_matrix
from ete3 import Tree

import logging

In [ ]:
d = 5
g = 10
vis = False
leaves_only = False
#var = 1.0

glm = Poisson_GLM(tree, g, d, vis, leaves_only, branch_lengths)
glm.simulate_latent()
#glm.simulate_ge()

***Visualizing t-SNE and PCA embeddings of the latent space***

In [ ]:
#z_embedded = TSNE(n_components=2).fit_transform(glm.z)
z_embedded = PCA(n_components=2).fit_transform(glm.z)

fig, ax = plt.subplots(figsize = (14, 8))

scatter = ax.scatter(z_embedded[:, 0],
           z_embedded[:, 1],
           cmap="Spectral")

plt.title('Simulated Latent Space')
plt.xlabel("z_1")
plt.ylabel("z_2")

plt.show()